In [1]:
import pandas as pd
import sed_eval

In [2]:
eval_path = "../dataset/metadata/eval/eval.csv"
df_eval = pd.read_csv(eval_path, sep="\t", header=0)
df_eval.head()

,filename,onset,offset,event_label
0,YSb0169-lqLs_0.000_9.000.wav,0.778,7.187,Dog
1,Y9OqtuFGCCR8_30.000_40.000.wav,0.000,0.302,Alarm_bell_ringing
2,Y9OqtuFGCCR8_30.000_40.000.wav,0.789,1.126,Alarm_bell_ringing
3,Y9OqtuFGCCR8_30.000_40.000.wav,1.567,1.939,Alarm_bell_ringing
4,Y9OqtuFGCCR8_30.000_40.000.wav,2.368,2.728,Alarm_bell_ringing


### Following cell is taken from evaluation_measures.py, but added here to be explicit

In [3]:
def get_event_list_current_file(df, fname):
    """
    Get list of events for a given filename
    :param df: pd.DataFrame, the dataframe to search on
    :param fname: the filename to extract the value from the dataframe
    :return: list of events (dictionaries) for the given filename
    """
    event_file = df[df["filename"].str.contains(fname)]
    if len(event_file) == 1:
        if pd.isna(event_file["event_label"].iloc[0]):
            event_list_for_current_file = [{"filename": fname}]
        else:
            event_list_for_current_file = event_file.to_dict('records')
    else:
        event_list_for_current_file = event_file.to_dict('records')

    return event_list_for_current_file

def event_based_evaluation_df(reference, estimated):
    """
    Calculate EventBasedMetric given a reference and estimated dataframe
    :param reference: pd.DataFrame containing "filename" "onset" "offset" and "event_label" columns which describe the
    reference events
    :param estimated: pd.DataFrame containing "filename" "onset" "offset" and "event_label" columns which describe the
    estimated events to be compared with reference
    :return: sed_eval.sound_event.EventBasedMetrics with the scores
    """
    
    evaluated_files = reference["filename"].unique()
    classes = []
    classes.extend(reference.event_label.dropna().unique())
    classes.extend(estimated.event_label.dropna().unique())
    event_based_metric = sed_eval.sound_event.EventBasedMetrics(
        event_label_list=classes,
        t_collar=0.200,
        percentage_of_length=0.2,
        empty_system_output_handling='zero_score'
    )

    for fname in evaluated_files:
        reference_event_list_for_current_file = get_event_list_current_file(reference, fname)
        estimated_event_list_for_current_file = get_event_list_current_file(estimated, fname)

        event_based_metric.evaluate(
            reference_event_list=reference_event_list_for_current_file,
            estimated_event_list=estimated_event_list_for_current_file,
        )
    return event_based_metric

In [4]:
df = pd.read_csv("eval_results_baseline.csv", sep="\t", header=0)
results = event_based_evaluation_df(df_eval, df)
print(results)

TypeError: __init__() got an unexpected keyword argument 'empty_system_output_handling'